In [1]:
import fourier
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import Counter
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(1337)

eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 50 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4

In [2]:
x = np.random.random(1024)
np.allclose(fourier.slow_dft(x) , np.fft.fft(x))

True

In [3]:
%timeit fourier.slow_dft(x)
%timeit np.fft.fft(x)

65.2 ms ± 2.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
16.3 μs ± 507 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [4]:
x = np.random.random(1024)
np.allclose(fourier.fft(x) , np.fft.fft(x))

True

In [5]:
%timeit fourier.slow_dft(x)
%timeit fourier.fft(x)
%timeit np.fft.fft(x)

64.1 ms ± 2.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.86 ms ± 12.9 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
15.6 μs ± 230 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [6]:
testinp = torch.rand(3,10,10)
shape = testinp.shape
print(testinp.shape)
out = torch.fft.fftn(testinp)
print(torch.view_as_real(out).shape)
out2 = torch.fft.fftn(out)
print(torch.view_as_real(out2).shape)
print(out2.real.shape)
print(torch.view_as_real(out2))
print(out2.real)

torch.Size([3, 10, 10])
torch.Size([3, 10, 10, 2])
torch.Size([3, 10, 10, 2])
torch.Size([3, 10, 10])
tensor([[[[ 23.4951,   0.0000],
          [ 77.8579,   0.0000],
          [ 52.8417,   0.0000],
          [209.0028,   0.0000],
          [175.5370,   0.0000],
          [  8.5961,   0.0000],
          [ 60.1165,   0.0000],
          [126.7314,   0.0000],
          [186.9294,   0.0000],
          [148.6850,   0.0000]],

         [[ 21.0111,   0.0000],
          [227.6316,   0.0000],
          [ 16.5947,   0.0000],
          [182.8949,   0.0000],
          [226.3931,   0.0000],
          [203.6695,   0.0000],
          [253.8217,   0.0000],
          [165.8158,   0.0000],
          [162.2757,   0.0000],
          [254.9681,   0.0000]],

         [[287.2652,   0.0000],
          [163.0083,   0.0000],
          [  6.5721,   0.0000],
          [ 29.3755,   0.0000],
          [156.7918,   0.0000],
          [178.3644,   0.0000],
          [222.7991,   0.0000],
          [108.3678,   0.0000]

In [7]:
class scaledDotProduct(nn.Module):
    '''
        Attention(Q, K, V ) = softmax( QK^T/√d_k)V 
    
    '''
    #Takes number of embedded, head_size, context length
    def __init__(self, embn, hdim, con_l, drop=0.0):

        super(scaledDotProduct, self).__init__()
        #dim is (d_k) when sqrt'd it is meant to counter small gradients in large sets of queries and keys
        self.k = nn.Linear(embn, hdim, bias=False)
        self.q = nn.Linear(embn, hdim, bias=False)
        self.v = nn.Linear(embn, hdim, bias=False)
        self.d_k = np.sqrt(hdim)

        self.register_buffer('mask', torch.tril(torch.ones(con_l,con_l)))
        #Simple drop out 
        self.drop = nn.Dropout(drop)

    def forward(self, x, ret_att=False):
        #batch X length X dim
        B,T,C = x.shape
        k = self.k(x)
        q = self.q(x)

        n = torch.matmul(q, k.transpose(-2,-1)) * k.shape[-1]**-0.5 #BxTxhdim
        n = n.masked_fill(self.mask[:T,:T]==0, float('-inf'))
        #Drop out referenced later in paper but not in original diagram
        att = self.drop(F.softmax(n, dim=-1))

        v = self.v(x)

        out = torch.matmul(att, v)
        if ret_att:
            return out, att 
        return out

In [8]:
class multiHeadedAttention(nn.Module):
    def __init__(self, n_heads, dims, embn, con_l, dropout=0.0):
        super(multiHeadedAttention, self).__init__()
        #d_k=d_v = dims/h

        self.n_heads = n_heads

        self.attn = nn.ModuleList([scaledDotProduct(embn, dims, con_l) for _ in range(n_heads)])
        #Final linear layer after concat and attention
        self.fc = nn.Linear(n_heads*dims, embn)

        self.drop = nn.Dropout(dropout)
        

    def forward(self, x):
        out = torch.cat([h(x) for h in self.attn], dim=-1)
        out = self.drop(self.fc(out))
        return out

In [9]:
class fftLayer(nn.Module):
    def __init__(self, dims, embn, dropout=0.0):
        super(fftLayer, self).__init__()
        #d_k=d_v = dims/h

        
        #Final linear layer after concat and attention
        self.fc = nn.Linear(dims, embn)

        self.drop = nn.Dropout(dropout)
        

    def forward(self, x):
        out = torch.fft.fftn(x)
        out = self.drop(self.fc(out.real))
        return out

In [10]:
class positionFeedFoward(nn.Module):
    def __init__(self, inp, hid, drop=0.0):
        super(positionFeedFoward, self).__init__()
        self.w1 = nn.Linear(inp,4*hid)
        self.w2 = nn.Linear(4*hid,inp)
        self.drop = nn.Dropout(drop)

    def forward(self, x):

        x = self.w2(F.relu(self.w1(x)))
        x = self.drop(x)

        return x

In [11]:
class Decoder(nn.Module):
    '''Combinds MultiHeadedAttention and FeeForward, three layers'''
    def __init__(self, nheads, embn, con_l, drop=0.0):
        super(Decoder, self).__init__()
        head_size = embn // nheads
        self.slf_attn = multiHeadedAttention(nheads, head_size,embn, con_l, dropout=drop)
        
        self.ffn = positionFeedFoward(embn, embn, drop=drop)

        self.norm1 = nn.LayerNorm(embn)
        self.norm2 = nn.LayerNorm(embn)

    def forward(self, x):
        x = x + self.slf_attn(self.norm1(x))
        x = x + self.ffn(self.norm2(x))

        return x

In [12]:
class fftDecoder(nn.Module):
    '''Combinds MultiHeadedAttention and FeeForward, three layers'''
    def __init__(self, dims, embn,  drop=0.0):
        super(fftDecoder, self).__init__()
        
        self.slf_fft = fftLayer(dims, embn,  dropout=drop)
        
        self.ffn = positionFeedFoward(embn, embn, drop=drop)

        self.norm1 = nn.LayerNorm(embn)
        self.norm2 = nn.LayerNorm(embn)

    def forward(self, x):
        x = x + self.slf_fft(self.norm1(x))
        x = x + self.ffn(self.norm2(x))

        return x

In [13]:
#Pytoch version adapted from here https://pub.aimind.so/creating-sinusoidal-positional-embedding-from-scratch-in-pytorch-98c49e153d6

class PosEncoding(nn.Module):
    def __init__(self, hid, n_pos=200):
        super(PosEncoding, self).__init__()

        self.register_buffer('pos_table', self._get_sinusoid_encoding_table(n_pos, hid))

    def _get_sinusoid_encoding_table(self, n_pos, hid):

        if hid %2 != 0:
            raise ValueError("Sinusoidal positional embedding cannot apply to odd token embedding dim={}".format(hid))
        
        positions = torch.arange(0,n_pos).unsqueeze_(1)
        embeds = torch.zeros(n_pos, hid)

        denom = torch.pow(10000, 2 * torch.arange(0, hid//2)/2)
        embeds[:, 0::2] = torch.sin(positions/denom)
        embeds[:, 1::2] = torch.cos(positions/denom)
        embeds = embeds.unsqueeze(0)

        return embeds
    
    def forward(self, x):
        return x + self.pos_table[:, :x.size(1)].clone().detach()

In [14]:
class languageModel(nn.Module):
    '''Decoder model'''
    def __init__(
            self, n_vocab, embn, n_layers, n_head, dropout=0.2 , con_l=200
    ):
        super(languageModel, self).__init__()
        self.con_l = con_l
        self.word_emb = nn.Embedding(n_vocab, embn)
        self.pos_enc = nn.Embedding(con_l, embn)
        attn_stack = [Decoder( n_head, embn, con_l, drop=dropout) for _ in range(n_layers//2)]
        fft_stack = [fftDecoder(embn,embn,dropout) for _ in range(n_layers//2)]
        self.stack = nn.Sequential(
            *[c for items in zip(attn_stack, fft_stack) for c in items]
        )
       
        self.layer_norm = nn.LayerNorm(embn)
        self.fc = nn.Linear(embn, n_vocab)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, x, tar=None):
        #batch, time
        B, T = x.shape

        tok = self.word_emb(x)
        pos = self.pos_enc(torch.arange(T, device=device))
        x = tok + pos
        x = self.stack(x)
        x = self.layer_norm(x)
        logits = self.fc(x)

        if tar is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            tar = tar.view(B*T)
            loss = F.cross_entropy(logits, tar)

        return logits, loss
    
    def generate(self, x, max_length):
        #x is a BxT array of in current context
        for _ in range(max_length):
            x_cond = x[:, -self.con_l:]
            logits, loss = self(x_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            x_next = torch.multinomial(probs, num_samples=1)
            x = torch.cat((x, x_next), dim=1)

        return x

In [15]:
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

torch.manual_seed(1337)

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits = model(X)[0]
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = Y.view(B*T)
            loss = F.cross_entropy(logits, targets)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [16]:
print(vocab_size)

65


In [17]:
model = languageModel(vocab_size,  384,12, 12, con_l=50
    )
m = model.to(device)
# print the number of parameters in the model
#print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
print(next(m.parameters()).is_cuda)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    #B, T, C = logits.shape
    #logits = logits.view(B*T, C)
    #targets = yb.view(B*T)
    #loss = F.cross_entropy(logits, targets)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_length=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

True
step 0: train loss 4.2570, val loss 4.2573
step 500: train loss 0.0922, val loss 0.0955
step 1000: train loss 0.0654, val loss 0.0678
step 1500: train loss 0.0580, val loss 0.0587
step 2000: train loss 0.0536, val loss 0.0540
step 2500: train loss 0.0507, val loss 0.0514
step 3000: train loss 0.0491, val loss 0.0489
step 3500: train loss 0.0478, val loss 0.0494
step 4000: train loss 0.0468, val loss 0.0483
step 4500: train loss 0.0471, val loss 0.0484
step 4999: train loss 0.0453, val loss 0.0468

o ee ee to  el re lee eeo  a  ee t r ae oe e ee ite  et er os. oe er  iee ia oe  e r t  oeei e e er ian s s elr i oee oe loe ii oe r mr io y e e ee o lo o iy eee er ate ie rdoooieo olr ooe ieee et io lee etee ae iot o leee io o ior eoe oee e eo os ier eee ilr ieo oete red  ooe lte oeet oe eo peea td ea i e ei eeee  oe  io   eee oe e y io ri e ea, e g le ee ieoe eo  ei lr iilt ii oo no oroee s ll  lee et ooo ee let. oe rs a i ee  let tlo it o e erl meer o e iee i eoed lr eee iole  ie r i 

In [18]:
sinp = "wherefore art thou"
sinp = torch.tensor(encode(sinp)).unsqueeze(0).to(device)
print(decode(model.generate(sinp, max_length=500)[0].tolist()))

wherefore art thoueee ee  ilr iaeeee eeee ee ee r ei y oe io oee ee  eo et ie eo  reee  lr d eo la e a  o ea e ea e ae  ae et lr  ialeroieie e tat olor lte r ee ooey d iet dtlr aeo are r lle o oeo ooo e re e ee ta o dro eeele c oe l ooe zeod io oo olee tc uo lo o erree lootooerood ooee oe rrro  aos  eee a le eo ai  oe  leed  oo o a e eremet ri o liee eeee ndd io to ieas  eoer er s iee ir oo dg  ae ee e oe splet iioe a holo ei oiald iola  ird ueortoy eo o d o eoloe eeit  iool et ro ie  ooo ee aet lleeeeeo ee it oe
